<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
import numpy as np

In [2]:
# input
X = np.array(([0, 0, 1],
              [0, 1, 1],
              [1, 0, 1],
              [0, 1, 0],
              [1, 0, 0],
              [1, 1, 1],
              [0, 0, 0]), dtype=float)

# output
y = np.array(([0],
              [1],
              [1],
              [1],
              [1],
              [0],
              [0]), dtype=float)

In [3]:
class NeuralNetwork:
    
    def __init__(self, inputs, hiddenNodes, outputNodes=1):
        # Set up Architecture of Neural Network
        self.inputs = inputs
        self.hiddenNodes =  hiddenNodes
        self.outputNodes = outputNodes

        self.inputWeights = np.random.randn(self.inputs, self.hiddenNodes)
        self.hiddenWeights = np.random.randn(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
        
    def sigmoidPrime(self, s):
        return s * (1-s)
    
    def feed_forward(self, X):
        
        # Weighted sum of inputs and hidden layer
        self.hiddenSum = np.dot(X, self.inputWeights)
        
        # Activation of the weighted sum
        self.hiddenActv = self.sigmoid(self.hiddenSum)
        
        # Weighted sum of hidden layer to output layer
        self.outputSum = np.dot(self.hiddenActv, self.hiddenWeights)
        
        # Final Activation of Output
        self.outputActv = self.sigmoid(self.outputSum)
        
        return(self.outputActv)
    
    def backward(self, X, y, o):
        
        self.o_error = y - o #error in output
        
        #Size of Adjustment from hidden => output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # apply derivate of sigmoid to error
        
        # z2 error: how much our input => hidden weights weights were off
        self.z2_error = self.o_delta.dot(self.hiddenWeights.T)
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.hiddenActv)
        
        #Adjustment hidden => output weights
        self.hiddenWeights += self.hiddenActv.T.dot(self.o_delta)
        
        #Adjustment input => hidden weights
        self.inputWeights += X.T.dot(self.z2_delta)
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [4]:
nn = NeuralNetwork(3,4)

nn.train(X, y)

In [5]:
for i in range(10000):
    if (i==0) | (i % 1000 == 0):
        print(f'----- --- EPOCH {i+1} --- -----')
        print('------------ \nInput\n', X,  '\n ------------')
        print('Correct Output \n', y,  '\n ------------')
        print('Predicted Output \n', str(nn.feed_forward(X)), '\n ------------')
        print("Loss \n", str(np.mean(np.square(y - nn.feed_forward(X)))), '\n ------------\n\n')
    nn.train(X,y)

----- --- EPOCH 1 --- -----
------------ 
Input
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]] 
 ------------
Correct Output 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]] 
 ------------
Predicted Output 
 [[0.6820262 ]
 [0.73785765]
 [0.70130615]
 [0.65384023]
 [0.62023117]
 [0.75742904]
 [0.58915831]] 
 ------------
Loss 
 0.2582790825669643 
 ------------


----- --- EPOCH 1001 --- -----
------------ 
Input
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]] 
 ------------
Correct Output 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]] 
 ------------
Predicted Output 
 [[0.01032903]
 [0.95056321]
 [0.94455794]
 [0.95298051]
 [0.95918283]
 [0.08358486]
 [0.01640153]] 
 ------------
Loss 
 0.0023938313866781647 
 ------------


----- --- EPOCH 2001 --- -----
------------ 
Input
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]] 
 ------------
Correct Output 
 [[0.]
 [1.]
 [1.]
 [1.]
 

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [6]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [7]:
# input image dimensions
img_rows, img_cols = 28, 28

In [8]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [9]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [10]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [11]:
y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [12]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [13]:
#use previous neural network class updated for new number of inputs
nn2 = NeuralNetwork(784, 1)

nn2.train(x_train, y_train)

MemoryError: 

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?